# **Cloud Computing Emissions Project**

In [15]:
import pandas as pd
import numpy as np
import requests
import os
import json

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
username = "tidyverse"
name_repo = "tidyverse"
token = "git_token"

## Collect the Worflow Runs from a chosen Repository

### Let us define a function to get 100 runs from a repository

In [18]:
def get_workflows_from_repo(token, user_name, name_repo, page_nb):

  url = f'https://api.github.com/repos/{user_name}/{name_repo}/actions/runs?per_page=100&page={page_nb}'

  cmd = 'curl -G -H f"Authorization: token {token_git}" {url}'
  os.system(cmd.format(token_git = token, url = url))

  headers = {'Authorization': f'token {token}'}
  response = requests.get(url, headers=headers)

  if response.status_code == 200:
      workflows_data = response.json()
  else:
      print(f"Échec de la requête avec le code d'état {response.status_code}")

  return pd.DataFrame(workflows_data['workflow_runs'])

In [19]:
df = get_workflows_from_repo(token, username, name_repo, 1)
df

,id,name,node_id,head_branch,head_sha,path,display_title,run_number,event,status,...,logs_url,check_suite_url,artifacts_url,cancel_url,rerun_url,previous_attempt_url,workflow_url,head_commit,repository,head_repository
0,7182346372,test-coverage,WFR_kwLOBAYhu88AAAABrBnohA,lintr,8e0fde20fcb45ddfaaa56e45b95b7f2de096968e,.github/workflows/test-coverage.yaml,Use .lintr to enforce tidyverse style,82,pull_request,completed,...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,None,https://api.github.com/repos/tidyverse/tidyver...,{'id': '8e0fde20fcb45ddfaaa56e45b95b7f2de09696...,"{'id': 67510715, 'node_id': 'MDEwOlJlcG9zaXRvc...","{'id': 727739570, 'node_id': 'R_kgDOK2Bssg', '..."
1,7182346369,R-CMD-check,WFR_kwLOBAYhu88AAAABrBnogQ,lintr,8e0fde20fcb45ddfaaa56e45b95b7f2de096968e,.github/workflows/R-CMD-check.yaml,Use .lintr to enforce tidyverse style,82,pull_request,completed,...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,None,https://api.github.com/repos/tidyverse/tidyver...,{'id': '8e0fde20fcb45ddfaaa56e45b95b7f2de09696...,"{'id': 67510715, 'node_id': 'MDEwOlJlcG9zaXRvc...","{'id': 727739570, 'node_id': 'R_kgDOK2Bssg', '..."
2,7182346354,pkgdown,WFR_kwLOBAYhu88AAAABrBnocg,lintr,8e0fde20fcb45ddfaaa56e45b95b7f2de096968e,.github/workflows/pkgdown.yaml,Use .lintr to enforce tidyverse style,71,pull_request,completed,...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,None,https://api.github.com/repos/tidyverse/tidyver...,{'id': '8e0fde20fcb45ddfaaa56e45b95b7f2de09696...,"{'id': 67510715, 'node_id': 'MDEwOlJlcG9zaXRvc...","{'id': 727739570, 'node_id': 'R_kgDOK2Bssg', '..."
3,7182346353,lint,WFR_kwLOBAYhu88AAAABrBnocQ,lintr,8e0fde20fcb45ddfaaa56e45b95b7f2de096968e,.github/workflows/lint.yaml,Use .lintr to enforce tidyverse style,2,pull_request,completed,...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,None,https://api.github.com/repos/tidyverse/tidyver...,{'id': '8e0fde20fcb45ddfaaa56e45b95b7f2de09696...,"{'id': 67510715, 'node_id': 'MDEwOlJlcG9zaXRvc...","{'id': 727739570, 'node_id': 'R_kgDOK2Bssg', '..."
4,7101667450,lint,WFR_kwLOBAYhu88AAAABp0rYeg,lintr,443e932b2be27c9babbca68f1566c987587e9b50,.github/workflows/lint.yaml,Use .lintr to enforce tidyverse style,1,pull_request,completed,...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,None,https://api.github.com/repos/tidyverse/tidyver...,{'id': '443e932b2be27c9babbca68f1566c987587e9b...,"{'id': 67510715, 'node_id': 'MDEwOlJlcG9zaXRvc...","{'id': 727739570, 'node_id': 'R_kgDOK2Bssg', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4256502504,R-CMD-check,WFR_kwLOBAYhu879tRLo,main,ce4a7ba700c62bc306e601271076fcfea399c094,.github/workflows/R-CMD-check.yaml,Increment version number to 2.0.0,66,push,completed,...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/tidyver...,https://api.github.com/repos/tidyverse/

### Let us define a function to get all workflow Runs from a repository

In [20]:
def get_all_workflows_from_repo(token, username, name_repo):

  df_final = pd.DataFrame()
  page_nb = 1
  while True:
    df = get_workflows_from_repo(token, username, name_repo, page_nb)

    if df.empty :
      break

    page_nb += 1
    df_final = pd.concat([df_final, df], axis = 0)

  return df_final

In [21]:
df = get_all_workflows_from_repo(token, username, name_repo)

In [ ]:
# Save Data containing runs in Drive file
df.to_csv(f'/content/drive/MyDrive/Data Mining Data/{name_repo}_wo_co2', index = False)

## Collect the jobs for the given Workflow Runs

### Let us define a function taking Runs from a Repo as parameter and sending all the jobs for these runs

In [22]:
def get_jobs_from_runs(token, username, name_repo, df_runs):

  df_jobs = pd.DataFrame()

  for i in df_runs['id']:

    url = f'https://api.github.com/repos/{username}/{name_repo}/actions/runs/{i}/jobs'

    cmd = f'curl -L \
      -H "Accept: application/vnd.github+json" \
      -H f"Authorization: Bearer {token}" \
      -H "X-GitHub-Api-Version: 2022-11-28" \
      {url}'

    os.system(cmd)

    headers = {'Authorization': f'token {token}'}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        run_id_jobs = response.json()
    else:
        print(f"Échec de la requête avec le code d'état {response.status_code}")
        continue

    df_jobs = pd.concat([df_jobs, pd.DataFrame(run_id_jobs['jobs'])], axis = 0)
  return df_jobs

In [23]:
pd.set_option('display.max_columns', 25)
df_jobs = get_jobs_from_runs(token, username, name_repo, df)
df_jobs.head()

,id,run_id,workflow_name,head_branch,run_url,run_attempt,node_id,head_sha,url,html_url,status,conclusion,created_at,started_at,completed_at,name,steps,check_run_url,labels,runner_id,runner_name,runner_group_id,runner_group_name
0,19558655750,7182346372,test-coverage,lintr,https://api.github.com/repos/tidyverse/tidyver...,1,CR_kwDOBAYhu88AAAAEjclnBg,8e0fde20fcb45ddfaaa56e45b95b7f2de096968e,https://api.github.com/repos/tidyverse/tidyver...,https://github.com/tidyverse/tidyverse/actions...,completed,success,2023-12-12T13:45:17Z,2023-12-12T13:45:24Z,2023-12-12T13:46:59Z,test-coverage,"[{'name': 'Set up job', 'status': 'completed',...",https://api.github.com/repos/tidyverse/tidyver...,[ubuntu-latest],13,GitHub Actions 13,2,GitHub Actions
0,19558655753,7182346369,R-CMD-check,lintr,https://api.github.com/repos/tidyverse/tidyver...,1,CR_kwDOBAYhu88AAAAEjclnCQ,8e0fde20fcb45ddfaaa56e45b95b7f2de096968e,https://api.github.com/repos/tidyverse/tidyver...,https://github.com/tidyverse/tidyverse/actions...,completed,success,2023-12-12T13:45:17Z,2023-12-12T13:45:25Z,2023-12-12T13:48:37Z,macos-latest (release),"[{'name': 'Set up job', 'status': 'completed',...",https://api.github.com/repos/tidyverse/tidyver...,[macos-latest],3,GitHub Actions 3,2,GitHub Actions
1,19558656399,7182346369,R-CMD-check,lintr,https://api.github.com/repos/tidyverse/tidyver...,1,CR_kwDOBAYhu88AAAAEjclpjw,8e0fde20fcb45ddfaaa56e45b95b7f2de096968e,https://api.github.com/repos/tidyverse/tidyver...,https://github.com/tidyverse/tidyverse/actions...,completed,success,2023-12-12T13:45:18Z,2023-12-12T13:45:27Z,2023-12-12T13:48:47Z,windows-latest (release),"[{'name': 'Set up job', 'status': 'completed',...",https://api.github.com/repos/tidyverse/tidyver...,[windows-latest],10,GitHub Actions 10,2,GitHub Actions
2,19558656825,7182346369,R-CMD-check,lintr,https://api.github.com/repos/tidyverse/tidyver...,1,CR_kwDOBAYhu88AAAAEjclrOQ,8e0fde20fcb45ddfaaa56e45b95b7f2de096968e,https://api.github.com/repos/tidyverse/tidyver...,https://github.com/tidyverse/tidyverse/actions...,completed,success,2023-12-12T13:45:19Z,2023-12-12T13:45:27Z,2023-12-12T13:49:55Z,windows-latest (3.6),"[{'name': 'Set up job', 'status': 'completed',...",https://api.github.com/repos/tidyverse/tidyver...,[windows-latest],9,GitHub Actions 9,2,GitHub Actions
3,19558657128,7182346369,R-CMD-check,lintr,https://api.github.com/repos/tidyverse/tidyver...,1,CR_kwDOBAYhu88AAAAEjclsaA,8e0fde20fcb45ddfaaa56e45b95b7f2de096968e,https://api.github.com/repos/tidyverse/tidyver...,https://github.com/tidyverse/tidyverse/actions...,completed,success,2023-12-12T13:45:19Z,2023-12-12T13:45:27Z,2023-12-12T13:52:44Z,windows-latest (4.1),"[{'name': 'Set up job', 'status': 'completed',...",https://api.github.com/repos/tidyverse/tidyver...,[windows-latest],11,GitHub Actions 11,2,GitHub Actions


In [24]:
df_jobs['started_at'] = pd.to_datetime(df_jobs['started_at'])
df_jobs['completed_at'] = pd.to_datetime(df_jobs['completed_at'])
df_jobs['duration'] = df_jobs['completed_at'] - df_jobs['started_at']
df_jobs['duration'] = pd.to_timedelta(df_jobs['duration'])/ pd.Timedelta(hours=1)

In [ ]:
# Save Data with Jobs to Drive file
df_jobs.to_csv('/content/drive/MyDrive/Data Mining Data/numpy_with_logs', index = False)

## Let us Create a function that computes Carbon Emissions from each Job using the Climatiq API.

In [26]:
df = df_jobs

In [27]:
df.columns

Index(['id', 'run_id', 'workflow_name', 'head_branch', 'run_url',
       'run_attempt', 'node_id', 'head_sha', 'url', 'html_url', 'status',
       'conclusion', 'created_at', 'started_at', 'completed_at', 'name',
       'steps', 'check_run_url', 'labels', 'runner_id', 'runner_name',
       'runner_group_id', 'runner_group_name', 'duration'],
      dtype='object')

In [30]:
apikey_climatiq = 'apikey_climatiq	'

In [31]:
def get_co2_emissions_from_jobs(df):
  climatiq_url = 'https://beta4.api.climatiq.io/compute/azure/cpu'
  co2_ems = []

  for i in range(len(df)):
    dur = df.iloc[i]['duration']

    data = {
    "cpu_count": 1,
    "region": "west_us",
    "duration": dur,
    "duration_unit": "h",
    }

    data = json.dumps(data)

    header = {
        'Authorization': f'Bearer {apikey_climatiq}',
    }

    response = requests.post(climatiq_url, headers=header, data=data)

    if response.status_code == 200:
        api_response = response.json()
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        print(response.text)
    co2_ems.append(api_response['co2e'])

  df['co2_emission'] = co2_ems
  df['unit'] = 'kg'

  return df

In [32]:
df = get_co2_emissions_from_jobs(df)

In [33]:
df

,id,run_id,workflow_name,head_branch,run_url,run_attempt,node_id,head_sha,url,html_url,status,conclusion,...,completed_at,name,steps,check_run_url,labels,runner_id,runner_name,runner_group_id,runner_group_name,duration,co2_emission,unit
0,19558655750,7182346372,test-coverage,lintr,https://api.github.com/repos/tidyverse/tidyver...,1,CR_kwDOBAYhu88AAAAEjclnBg,8e0fde20fcb45ddfaaa56e45b95b7f2de096968e,https://api.github.com/repos/tidyverse/tidyver...,https://github.com/tidyverse/tidyverse/actions...,completed,success,...,2023-12-12 13:46:59+00:00,test-coverage,"[{'name': 'Set up job', 'status': 'completed',...",https://api.github.com/repos/tidyverse/tidyver...,[ubuntu-latest],13,GitHub Actions 13,2,GitHub Actions,0.026389,0.000017,kg
0,19558655753,7182346369,R-CMD-check,lintr,https://api.github.com/repos/tidyverse/tidyver...,1,CR_kwDOBAYhu88AAAAEjclnCQ,8e0fde20fcb45ddfaaa56e45b95b7f2de096968e,https://api.github.com/repos/tidyverse/tidyver...,https://github.com/tidyverse/tidyverse/actions...,completed,success,...,2023-12-12 13:48:37+00:00,macos-latest (release),"[{'name': 'Set up job', 'status': 'completed',...",https://api.github.com/repos/tidyverse/tidyver...,[macos-latest],3,GitHub Actions 3,2,GitHub Actions,0.053333,0.000035,kg
1,19558656399,7182346369,R-CMD-check,lintr,https://api.github.com/repos/tidyverse/tidyver...,1,CR_kwDOBAYhu88AAAAEjclpjw,8e0fde20fcb45ddfaaa56e45b95b7f2de096968e,https://api.github.com/repos/tidyverse/tidyver...,https://github.com/tidyverse/tidyverse/actions...,completed,success,...,2023-12-12 13:48:47+00:00,windows-latest (release),"[{'name': 'Set up job', 'status': 'completed',...",https://api.github.com/repos/tidyverse/tidyver...,[windows-latest],10,GitHub Actions 10,2,GitHub Actions,0.055556,0.000036,kg
2,19558656825,7182346369,R-CMD-check,lintr,https://api.github.com/repos/tidyverse/tidyver...,1,CR_kwDOBAYhu88AAAAEjclrOQ,8e0fde20fcb45ddfaaa56e45b95b7f2de096968e,https://api.github.com/repos/tidyverse/tidyver...,https://github.com/tidyverse/tidyverse/actions...,completed,success,...,2023-12-12 13:49:55+00:00,windows-latest (3.6),"[{'name': 'Set up job', 'status': 'completed',...",https://api.github.com/repos/tidyverse/tidyver...,[windows-latest],9,GitHub Actions 9,2,GitHub Actions,0.074444,0.000048,kg
3,19558657128,7182346369,R-CMD-check,lintr,https://api.github.com/repos/tidyverse/tidyver...,1,CR_kwDOBAYhu88AAAAEjclsaA,8e0fde20fcb45ddfaaa56e45b95b7f2de096968e,https://api.github.com/repos/tidyverse/tidyver...,https://github.com/tidyverse/tidyverse/actions...,completed,success,...,2023-12-12 13:52:44+00:00,windows-latest (4.1),"[{'name': 'Set up job', 'status': 'completed',...",https://api.github.com/repos/tidyverse/tidyver...,[windows-latest],11,GitHub Actions 11,2,GitHub Actions,0.121389,0.000079,kg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,9743297000,3564477841,Commands,main,https://api.github.com/repos/tidyverse/tidyver...,1,CR_kwDOBAYhu88AAAACRL7p6A,aeabcde8c6ae435f16b5173682d5667d292829fb,https://api.github.com/repos/tidyverse/tidyver...,https://github.com/tidyverse/tidyverse/actions...,completed,skipped,...,2022-11-28 11:51:59+00:00,document,[],https://api.github.com/repos/tidyverse/tidyver...,[],None,None,None,None,0.000000,0.000000,kg
0,9743290561,3564475573,Commands,main,https://api.github.com/repos/tidyverse/tidyver...,1,CR_kwDOBAYhu88AAAACRL7QwQ,aeabcde8c6ae435f16b5173682d5667d292829fb,https://api.github.com/repos/tidyverse/tidyver...,https://github.com/tidyverse/tidyverse/actions...,completed,skipped,...,2022-11-28 11:51:37+00:00,style,[],https://api.github.com/repos/tidyverse/tidyver...,[],None,None,None,None,0.000000,0.000000,kg
1,9743290824,3564475573,Commands,main,https://api.github.com/repos/tidyverse/tidyver...,1,CR_kwDOBAYhu88AAAACRL7RyA,aeabcde8c6ae435f16b5173682d5667d292829fb,https://api.github.com/repos/tidyverse/tidyver...,https://github.com/tidyverse/tidyverse/actions...,completed,skipped,...,2022-11-28 11:51:38+00

In [14]:
# Save final data to Drive file
df.to_csv(f'/content/drive/MyDrive/Data Mining Data/pandas_data_with_co2.csv')